<a href="https://colab.research.google.com/github/baoduy2048/rag/blob/main/base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers faiss-cpu langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.1/476.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.1.3
    Uninstalling langchain-core-1.1.3:
      Successfully uninstalled langchain-core-1.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-c

In [ ]:
!pip install pymupdf langchain

In [ ]:
import fitz  # PyMuPDF
import re
import os
import logging

# Thiết lập Logging để theo dõi quá trình xử lý
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.FileHandler("ingestion.log"), logging.StreamHandler()]
)

class LegalDataIngestor:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path
        self.file_name = os.path.basename(pdf_path)

    def clean_text(self, text):
        """Lọc bỏ các ký tự rác và định dạng thừa"""
        # Loại bỏ khoảng trắng thừa ở đầu/cuối dòng
        text = "\n".join([line.strip() for line in text.split("\n")])
        # Loại bỏ nhiều dấu xuống dòng liên tiếp
        text = re.sub(r'\n{3,}', '\n\n', text)
        return text

    def is_header_footer(self, block_text, page_num, total_pages):
        """
        Dùng Regex để nhận diện số trang hoặc tiêu đề lặp lại
        (Ví dụ: 'Văn bản hợp nhất...', 'Trang 1/200')
        """
        patterns = [
            r'Trang\s+\d+',            # Ví dụ: Trang 1
            r'^\d+$',                  # Chỉ có số (thường là số trang)
            r'VĂN BẢN HỢP NHẤT',       # Tiêu đề lặp lại
            r'BỘ TƯ PHÁP'              # Cơ quan ban hành lặp lại
        ]
        for p in patterns:
            if re.search(p, block_text, re.IGNORECASE):
                return True
        return False

    def process(self, output_txt_path):
        logging.info(f"Bắt đầu xử lý file: {self.file_name}")
        structured_data = []
        full_clean_text = ""

        try:
            doc = fitz.open(self.pdf_path)
            total_pages = len(doc)

            for page_num in range(total_pages):
                page = doc[page_num]
                # Lấy text theo từng khối (blocks) để dễ kiểm soát bối cảnh
                blocks = page.get_text("blocks")

                page_content = []
                for b in blocks:
                    block_text = b[4] # Nội dung text nằm ở index 4

                    # Loại bỏ Header/Footer
                    if not self.is_header_footer(block_text, page_num + 1, total_pages):
                        page_content.append(block_text)

                # Gộp text trang đã sạch
                clean_page_text = self.clean_text(" ".join(page_content))

                # Lưu trữ kèm metadata
                structured_data.append({
                    "page": page_num + 1,
                    "content": clean_page_text
                })

                full_clean_text += f"\n--- TRANG {page_num + 1} ---\n{clean_page_text}\n"

            # Xuất ra file text trung gian
            with open(output_txt_path, "w", encoding="utf-8") as f:
                f.write(full_clean_text)

            logging.info(f"Hoàn thành! Đã lưu file text sạch tại: {output_txt_path}")
            return structured_data

        except Exception as e:
            logging.error(f"Lỗi trong quá trình Ingestion: {str(e)}")
            return None

# --- CHẠY THỬ MODULE 1 ---
if __name__ == "__main__":
    ingestor = LegalDataIngestor("155-vbhn-vpqh.pdf")
    # Trích xuất dữ liệu có cấu trúc
    data = ingestor.process("he_thong_A_cleaned.txt")

    if data:
        print(f"Đã trích xuất xong {len(data)} trang.")
        print("Ví dụ nội dung trang 1:", data[0]['content'][:200], "...")

Đã trích xuất xong 134 trang.
Ví dụ nội dung trang 1: CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM
Độc lập - Tự do - Hạnh phúc

LUẬT
SỞ HỮU TRÍ TUỆ

Luật Sở hữu trí tuệ số 50/2005/QH11 ngày 29 tháng 11 năm 2005 của Quốc
hội, có hiệu lực kể từ ngày 01 tháng 7 năm 2 ...


In [ ]:
!pip install -U langchain-text-splitters langchain-community langchain-huggingface

In [ ]:
import os
# Sử dụng thư viện mới để tránh ModuleNotFoundError
try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except ImportError:
    from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

class LegalVectorStore:
    def __init__(self, model_name="bkai-foundation-models/vietnamese-bi-encoder"):
        print("--- Đang tải mô hình Embedding tiếng Việt (BKAI) ---")
        # Kiểm tra GPU trên Colab
        import torch
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Sử dụng thiết bị: {device}")

        self.embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={'device': device}
        )
        self.vector_db = None

    def create_chunks(self, input_txt_path):
        print("--- Giai đoạn 2.1: Chia nhỏ văn bản (200 từ) ---")
        with open(input_txt_path, "r", encoding="utf-8") as f:
            text = f.read()

        # Chunk size 1000 ký tự thường tương đương ~200 từ tiếng Việt
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=150,
            separators=["\n\n", "\n", ". ", " ", ""]
        )

        chunks = text_splitter.split_text(text)
        print(f"Tổng số chunks tạo ra: {len(chunks)}")
        return chunks

    def build_and_save(self, chunks, store_path="faiss_baseline"):
        print(f"--- Giai đoạn 2.2: Embedding và Lưu trữ ---")
        self.vector_db = FAISS.from_texts(chunks, self.embeddings)
        self.vector_db.save_local(store_path)
        print(f"Hệ thống A đã sẵn sàng tại: {store_path}")

# Thực thi
vector_tool = LegalVectorStore()
if os.path.exists("he_thong_A_cleaned.txt"):
    chunks = vector_tool.create_chunks("he_thong_A_cleaned.txt")
    vector_tool.build_and_save(chunks)
else:
    print("Lỗi: Không tìm thấy file he_thong_A_cleaned.txt từ Module 1!")

--- Đang tải mô hình Embedding tiếng Việt (BKAI) ---
Sử dụng thiết bị: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

bpe.codes: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

--- Giai đoạn 2.1: Chia nhỏ văn bản (200 từ) ---
Tổng số chunks tạo ra: 512
--- Giai đoạn 2.2: Embedding và Lưu trữ ---
Hệ thống A đã sẵn sàng tại: faiss_baseline


In [ ]:
!pip install -U langchain

In [ ]:
!pip install -U langchain-core langchain-huggingface langchain-community langchain-google-genai

In [ ]:
import os
import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

class LegalChatSystem:
    def __init__(self, vector_store_path="faiss_baseline"):
        print("--- Đang khởi tạo Hệ thống A (Kiến trúc LCEL 2025) ---")

        # 1. Khởi tạo Embedding
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.embeddings = HuggingFaceEmbeddings(
            model_name="bkai-foundation-models/vietnamese-bi-encoder",
            model_kwargs={'device': device}
        )

        # 2. Tải Vector Database
        self.vector_db = FAISS.load_local(
            vector_store_path,
            self.embeddings,
            allow_dangerous_deserialization=True
        )

        # 3. Cấu hình Gemini
        os.environ["GOOGLE_API_KEY"] = "DÁN_API_KEY_CỦA_BẠN_VÀO_ĐÂY"
        self.llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

    def format_docs(self, docs):
        return "\n\n".join(doc.page_content for doc in docs)

    def get_answer(self, query):
        # Đảm bảo query sạch
        query = str(query).strip()

        template = """Bạn là chuyên gia về Luật Sở hữu trí tuệ Việt Nam.
        Ngữ cảnh: {context}
        Câu hỏi: {question}
        Trả lời:"""

        prompt = PromptTemplate.from_template(template)
        retriever = self.vector_db.as_retriever(search_kwargs={"k": 5})

        # Sử dụng invocation trực tiếp để kiểm soát dữ liệu đầu vào
        rag_chain = (
            {"context": retriever | self.format_docs, "question": RunnablePassthrough()}
            | prompt
            | self.llm
            | StrOutputParser()
        )

        response = rag_chain.invoke(query)
        # Lấy sources riêng để tránh lỗi metadata khi invoke
        sources = retriever.invoke(query)

        return response, sources

# --- KHỞI TẠO ---
try:
    qa_system = LegalChatSystem()
    print("✅ Hệ thống A đã sẵn sàng với kiến trúc LCEL!")
except Exception as e:
    print(f"❌ Vẫn gặp lỗi: {e}")

--- Đang khởi tạo Hệ thống A (Kiến trúc LCEL 2025) ---
✅ Hệ thống A đã sẵn sàng với kiến trúc LCEL!


In [ ]:
import os
import locale

# Ép kiểu encoding
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# Xóa các biến môi trường có thể gây lỗi cho httpx
bad_keys = ['HTTP_PROXY', 'HTTPS_PROXY', 'ALL_PROXY', 'http_proxy', 'https_proxy', 'all_proxy']
for key in bad_keys:
    if key in os.environ:
        del os.environ[key]

# Thiết lập NO_PROXY để httpx không kiểm tra hệ thống
os.environ['no_proxy'] = '*'

# Đảm bảo API Key sạch (Thay bằng key của bạn)
MY_API_KEY = "GOOGLE_API_KEY".strip()
os.environ["GOOGLE_API_KEY"] = MY_API_KEY

In [ ]:
import pandas as pd
import time
import locale

class LegalEvaluator:
    def __init__(self, qa_system):
        self.qa_system = qa_system
        self.results = []

    def run_benchmark(self, test_cases):
        print("--- Bắt đầu đánh giá ---")
        self.results = []

        for i, case in enumerate(test_cases):
            q = case['question']
            gt = case['ground_truth']

            try:
                # Không dùng print trực tiếp biến q nếu q chứa ký tự lạ gây lỗi console
                print(f"Đang xử lý câu {i+1}...")

                start_time = time.time()
                # Ép query về string chuẩn UTF-8
                safe_query = str(q).encode('utf-8').decode('utf-8')
                answer, sources = self.qa_system.get_answer(safe_query)
                latency = time.time() - start_time

                self.results.append({
                    "Question": q,
                    "Baseline_Answer": answer,
                    "Ground_Truth": gt,
                    "Latency": f"{latency:.2f}s"
                })
                print(f"✅ Câu {i+1} thành công")
            except Exception as e:
                # In ra lỗi cụ thể để debug
                print(f"❌ Lỗi tại câu {i+1}: {str(e)}")

        df = pd.DataFrame(self.results)
        df.to_csv("benchmark_system_A.csv", index=False, encoding="utf-8-sig")
        return df

# --- THIẾT LẬP BỘ CÂU HỎI THỬ NGHIỆM (BENCHMARK DATASET) ---
# Bạn nên chọn những câu hỏi mà bạn biết chắc chắn đáp án trong luật
test_queries = [
    {
        "question": "Thời hạn bảo hộ quyền tác giả đối với tác phẩm văn học là bao lâu?",
        "ground_truth": "Suốt cuộc đời tác giả và 50 năm tiếp theo năm tác giả chết."
    },
    {
        "question": "Sáng chế được bảo hộ nếu đáp ứng những điều kiện nào?",
        "ground_truth": "Có tính mới, có trình độ sáng tạo và có khả năng áp dụng công nghiệp."
    },
    {
        "question": "Hành vi nào bị coi là xâm phạm quyền đối với nhãn hiệu?",
        "ground_truth": "Sử dụng dấu hiệu trùng hoặc tương tự với nhãn hiệu đã được bảo hộ..."
    }
]

# --- CHẠY ĐÁNH GIÁ ---
evaluator = LegalEvaluator(qa_system)
benchmark_df = evaluator.run_benchmark(test_queries)

# Hiển thị kết quả ngay trên Colab
benchmark_df.head()

--- Bắt đầu đánh giá 3 câu hỏi ---
Đang xử lý câu 1...
Lỗi tại câu 1: 'ascii' codec can't encode character '\xc1' in position 1: ordinal not in range(128)
Đang xử lý câu 2...
Lỗi tại câu 2: 'ascii' codec can't encode character '\xc1' in position 1: ordinal not in range(128)
Đang xử lý câu 3...
Lỗi tại câu 3: 'ascii' codec can't encode character '\xc1' in position 1: ordinal not in range(128)


""
